## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN','NAME'], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
for column in application_df.columns:
    uniquevals = application_df[column].value_counts().unique()
    print(f"{column}: {len(uniquevals)}", )

APPLICATION_TYPE: 15
AFFILIATION: 6
CLASSIFICATION: 33
USE_CASE: 5
ORGANIZATION: 4
STATUS: 2
INCOME_AMT: 9
SPECIAL_CONSIDERATIONS: 2
ASK_AMT: 4
IS_SUCCESSFUL: 2


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T13','T12','T2','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1248        1
C6100        1
C1820        1
C1900        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
values = application_df['CLASSIFICATION'].value_counts()
i = 0
for value in values:
    if value > 1:
        print(values.index[i], value)
    i += 1

C1000 17326
C2000 6074
C1200 4837
C3000 1918
C2100 1883
C7000 777
C1700 287
C4000 194
C5000 116
C1270 114
C2700 104
C2800 95
C7100 75
C1300 58
C1280 50
C1230 36
C1400 34
C7200 32
C2300 32
C1240 30
C8000 20
C7120 18
C1500 16
C6000 15
C1800 15
C1250 14
C8200 11
C1238 10
C1278 10
C1237 9
C1235 9
C7210 7
C1720 6
C2400 6
C4100 6
C1257 5
C1600 5
C1260 3
C2710 3
C0 3
C3200 2
C1256 2
C1246 2
C1234 2
C1267 2


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
values = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = [values.index[i] for i in range(len(values)) if values[i] < 1883]
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

C:\Users\Leo\AppData\Local\Temp\ipykernel_22812\1951361344.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  classifications_to_replace = [values.index[i] for i in range(len(values)) if values[i] < 1883]


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df
dummies = pd.get_dummies(application_df[['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','SPECIAL_CONSIDERATIONS', 'INCOME_AMT']], dtype=int)
application_df.drop(['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','SPECIAL_CONSIDERATIONS', 'INCOME_AMT'], axis=1, inplace=True)


In [10]:
merge = pd.merge(application_df, dummies, on=application_df.index)
merge.drop('key_0', axis=1, inplace=True)
merge

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,SPECIAL_CONSIDERATIONS_Y,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [11]:
merge.to_csv('processed_dat.csv', index=False)

In [12]:
# Split our preprocessed data into our features and target arrays
y = merge['IS_SUCCESSFUL']
X = merge.drop('IS_SUCCESSFUL', axis=1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Pre-Tuning: creating `model.h5`

In [159]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_feat = 44

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=input_feat))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

c:\Users\Leo\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 80)             │         3,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 50)             │         4,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,701 (30.08 KB)

 Trainable params: 7,701 (30.08 KB)

 Non-trainable params: 0 (0.00 B)

In [160]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [161]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step - accuracy: 0.7185 - loss: 0.5787
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 757us/step - accuracy: 0.7255 - loss: 0.5585
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 745us/step - accuracy: 0.7287 - loss: 0.5551
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 755us/step - accuracy: 0.7300 - loss: 0.5503
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 741us/step - accuracy: 0.7294 - loss: 0.5521
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 749us/step - accuracy: 0.7273 - loss: 0.5512
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 755us/step - accuracy: 0.7357 - loss: 0.5469
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 769us/step - accuracy: 0.7383 - loss: 0.5439
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 757us/step - accuracy: 0.7364 - loss: 0.5436
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 749us/step - accuracy: 0.7353 - loss: 0.5457
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 749us/step - accuracy: 0.7345 - loss: 0.5459
Epoch 12/100
804/80

In [162]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7282 - loss: 0.5602
Loss: 0.5602476596832275, Accuracy: 0.7281632423400879


In [163]:
# Export our model to HDF5 file
nn.save('model.h5')

# Post-Tuning: creating `model_v2.h5`

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_feat = 44

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=34, activation='relu', input_dim=input_feat))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=78, activation='relu'))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=21, activation='relu'))

# Final hidden layer
nn2.add(tf.keras.layers.Dense(units=15, activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

c:\Users\Leo\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 34)             │         1,530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 78)             │         2,730 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 21)             │         1,659 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │           330 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,265 (24.47 KB)

 Trainable params: 6,265 (24.47 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
nn2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
fit_model2 = nn2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 885us/step - accuracy: 0.6855 - loss: 0.6033
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 843us/step - accuracy: 0.7290 - loss: 0.5549
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 842us/step - accuracy: 0.7289 - loss: 0.5560
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 841us/step - accuracy: 0.7354 - loss: 0.5461
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 865us/step - accuracy: 0.7372 - loss: 0.5459
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 841us/step - accuracy: 0.7322 - loss: 0.5516
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 859us/step - accuracy: 0.7325 - loss: 0.5480
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 850us/step - accuracy: 0.7315 - loss: 0.5492
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 868us/step - accuracy: 0.7326 - loss: 0.5485
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 852us/step - accuracy: 0.7304 - loss: 0.5491
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 846us/step - accuracy: 0.7365 - loss: 0.5443
Epoch 12/100
804/80

In [17]:
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7311 - loss: 0.5651
Loss: 0.5650750994682312, Accuracy: 0.7310787439346313


In [18]:
nn2.save('model_v2.h5')